In [1]:
import numpy as np
import math
import matplotlib.pyplot as plt
import matplotlib.animation
from scipy.sparse        import diags_array
from scipy.sparse.linalg import spsolve
%matplotlib inline
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
display(HTML("<style>.output_result { width:90% !important; }</style>"))
display(HTML(""" <style>.output_png { display: table-cell; text-align: center; vertical-align: middle; }</style> """))
display(HTML("<style> div.text_cell_render { line-height: 2.0; } </style>"))    # Adjust linespread

In [47]:
def plotting(U, x, t, exact_handle, D):
    # grid spacing
    dx = x[1] - x[0]

    markers = ['x','s','d','+','o','.']
    colours = ['xkcd:salmon','xkcd:golden rod','xkcd:teal',
               'xkcd:purple','xkcd:slate','xkcd:silver']
    disp_times = [1, N//20, N//10, N//5, N//2, N]

    fig, ax = plt.subplots(1, 1, figsize=(12, 5))
    ax.scatter([0], [0], color='white', label=r'Exact $u(x,t)$', lw=0)
    for i, idx_t in enumerate(disp_times):
        lbl = ('t = {0:.0e}'.format(t[idx_t])
               if 0 < t[idx_t] < T else
               't = {0:.0f}'.format(t[idx_t]))
        ax.plot(x, exact_handle(x, t[idx_t], D(x, alpha=0.1)),
                color=colours[i], label=lbl)

    ax.scatter([0], [0], color='white',
               label=r'Numerical $U(x,t)$', lw=0)
    for i, idx_t in enumerate(disp_times):
        lbl = ('t = {0:.0e}'.format(t[idx_t])
               if 0 < t[idx_t] < T else
               't = {0:.0f}'.format(t[idx_t]))
        ax.plot(x[::2], U[idx_t][::2],
                linestyle='', marker=markers[i],
                color=colours[i], label=lbl)

    ax.legend(ncols=2, loc='center',
              bbox_to_anchor=(1.2, 0.5), title='Comparison')
    # show full domain
    ax.set_xlim(x.min() - dx, x.max() + dx)
    # show a line at x = 0.25
    ax.axvline(x=np.ceil(len(x)/2), color='gray', linestyle='--', lw=1)

    plt.xlabel("Space (x)")
    plt.ylabel("Solution")
    plt.show()

In [45]:
# --- Now we want to implement the screen ---

# To do this we need to model the screen to the right and left (assuming our screen starts to the right x0)
# On the right we replicate the toy model but with right boundary condition u_x(screen, t)=0.01 (or some small number)
# On the left of the screen we need to solve the diffusion equation again but wiht the initial condition of x_0=0
# and have boundary conditions of u_x(screen, t)=-0.01 such that is the negative of the right side of the screen,
# and have a zero BC at the end of the room L

# --- To investigate further ---

# The use of a function of x,t as the boudnary condition at the screen
# Implementation of advection to the model 

# ---- What we want to find ----

# Find an optimal aplha (diffusivity consant inside screen)

In [ ]:
# ** Toy Model inital contion and exact solution ** #
def D(x, alpha):
    D_air = 1.0
    D_screen = alpha
    x_screen = np.ceil(len(x)/2)
    D = np.zeros_like(x)
    for i in range(len(x)):
        D[i] = D_air
    D[x_screen] = D_screen
    return D


def f(x):
    x0 = x[len(x)//2]
    epsilon = 0.05 
    
    U0 = np.zeros_like(x)
    mask = np.abs(x - x0) <= epsilon
    U0[mask] = 1 / (2 * epsilon) 
    return U0

def U_exact(x, t, D):
    if t < 1e-10:
        return f(x)
    
    u_exact = 1.0 / (2 * np.sqrt(np.pi * D * t)) * np.exp(-x**2 / (4 * D * t))
    
    return u_exact

N = 2000      # number of timesteps
T = 1      # final time 
t = np.linspace(1/N, T, N+1)      # Time grid

M = 100    # number of spaces in x
x = np.linspace(-2, 2, M + 1)    # Space grid

dx = x[1] - x[0] 
dt = t[1] - t[0]

C = dt/(dx**2)
U = np.zeros((N+1,M+1))
U[0] = f(x)

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices